## Importing the libraries

In [ ]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
import warnings
import gc
warnings.filterwarnings("ignore")
date = pd.Timestamp.now().strftime('%m/%d/%Y')
import re
gc.enable()

# Import Datasets

In [ ]:
vdata=pd.read_csv('2020VAERSData.csv', encoding='latin1', dtype={'BIRTH_DEFECT':object},low_memory=False)
vvax = pd.read_csv('2020VAERSVAX.csv',encoding='latin1',low_memory=False)
vsymp = pd.read_csv('2020VAERSSYMPTOMS.csv',encoding='latin1',low_memory=False)

# Dropping columns so you can speed up data 

In [ ]:
dp =[2,4,6,8,10]
vsymp.drop(vsymp.columns[dp],axis=1, inplace=True)
c = [7,12,15,23]
vdata.drop(vdata.columns[c],axis=1, inplace=True)

# Merging datasets

In [ ]:
v1 = vdata.merge(vvax, on='VAERS_ID', how='outer')
VAERSM = v1.merge(vsymp, on ="VAERS_ID", how='outer')

# Have a database which only has COVID Vaccines 

In [ ]:
VAERSM = VAERSM.query('VAX_TYPE== "COVID19"')

# Exporting merged data as a single CSV File 

In [ ]:
VAERSM.to_csv("VDUP.csv", index=False)

# Deleteing old datasets 

In [ ]:
del vdata
del vvax
del vsymp
del VAERSM

# Check new dataset

In [ ]:
VDUP = pd.read_csv('VDUP.csv',encoding='latin1',memory_map=True, low_memory=False)
test = VDUP.duplicated(subset=["VAERS_ID"])
print(test.value_counts())
del test

False    193696
True     105895
dtype: int64


# Isolating by vaccine type 

In [ ]:
VDUP.query('VAX_TYPE=="COVID19"',inplace=True) 
gc.collect()

73

# Copying dataset into new variable 

In [ ]:
fullframe = VDUP.copy(deep=True) 

# Removing duplicate VAERS IDs

In [ ]:
nodupids =  VDUP.drop_duplicates(subset=['VAERS_ID'], keep='first')

In [ ]:
del VDUP

In [ ]:
print(len(fullframe),len(nodupids))

299591 193696


# Expanding column to 5000 characters 

In [ ]:
pd.options.display.max_colwidth = 5000

In [ ]:
nodupids[['VAERS_ID','SYMPTOM_TEXT']].head(3)

,VAERS_ID,SYMPTOM_TEXT
0,1996874,"SUSPECTED CLINICAL VACCINATION FAILURE; SUSPECTED COVID-19 INFECTION; This spontaneous potential legal report received from a patient concerned a patient of unspecified age and sex. Initial information was processed along with the additional information received from a regulatory authority on 30-DEC-2021. The patient's height, and weight were not reported. No past medical history or concurrent conditions were reported. The patient received covid-19 vaccine ad26.cov2.s (dose number in series: 1) (suspension for injection, route of admin not reported, batch number: Unknown and expiry: UNKNOWN) dose, start therapy date were not reported, administered 1 in total for prophylactic vaccination. The batch number was not reported. The Company is unable to perform follow-up to request batch/lot numbers. No concomitant medications were reported. It was reported that on an unspecified dates, the patient caught covid twice(coded as suspected covid-19 infection and suspected clinical vaccination failure) and reporter also stated that ""If anyone is doing a class action lawsuit against these clowns, hit me up. I want in"". The action taken with covid-19 vaccine ad26.cov2.s was not applicable. The outcome of the suspected covid-19 infection and suspected clinical vaccination failure was not reported. This report was serious (Other Medically Important Condition). This report was associated with a product quality complaint 90000209391. The suspected product quality complaint has been confirmed to be voided(did not meet RA criteria) (as reported complaint was covid with no symptoms and test performed therefore did not meet RA criteria, hence was confirmed as voided) based on the RA evaluation/investigation performed.; Sender's Comments: V0: 20211263430 -Covid-19 vaccine ad26.cov2.s-Suspected clinical Vaccination failure. This event(s) is considered not related. The event(s) has a compatible/suggestive temporal relationship, is unlabeled, and has unknown scientific plausibility. There are other factors more likely to be associated with the event(s) than the drug. Specifically: SPECIAL SITUATIONS"
1,1996875,"SUSPECTED CLINICAL VACCINATION FAILURE; SUSPECTED COVID-19 INFECTION; This spontaneous report received from a patient concerned a male of unspecified age. The patient's height, and weight were not reported. No past medical history or concurrent conditions were reported. The patient received covid-19 vaccine ad26.cov2.s (dose number in series was 1) (suspension for injection, route of admin not reported, batch number: 204A21A, expiry: UNKNOWN) dose was not reported, 1 total, administered on 11-MAY-2021 in left arm for prophylactic vaccination. No concomitant medications were reported. On 28-DEC-2021, the patient experienced feeling tired, had headaches, fever, stuffy nose, chills, body aches, nasal congestion and cough (suspected covid-19 infection). On 30-DEC-2021, Laboratory data included: COVID-19 antigen test (NR: not provided) positive (suspected clinical vaccination failure). The action taken with covid-19 vaccine ad26.cov2.s was not applicable. The outcome of the suspected clinical vaccination failure and suspected covid-19 infection was not reported. This report was serious (Other Medically Important Condition). This case is associated to product quality complaint.; Sender's Comments: V0- 20211266010-Covid-19 vaccine ad26.CoV-2.s- Suspected clinical vaccination failure. This event is considered not related. The event has a compatible/suggestive temporal relationship, is unlabeled, and has unknown scientific plausibility. There are other factors more likely to be associated with the event than the drug. Specifically: SPECIAL SITUATIONS"
2,1996876,"Irregular menstrual cycle.; period extremely heavy; bad cramping; This is a spontaneous report received from a contactable reporter(s) (Other HCP). The reporter is the patient. A 34 year-old female patient (not pregnant) received bnt162b2 (BNT162B2), administer

# Creating a variable for people who have died by the COVID Vaccines 

In [ ]:
D_nodupids = nodupids.query('DIED=="Y"')

# Creating a variable for people who have died by the COVID Vaccines within 1 day or less

In [ ]:
zero_d = D_nodupids.query('NUMDAYS <= 1',)
zero_d.replace({np.nan:'N' },regex=True,inplace=True)
d_count =len(zero_d)

# Wrapping text

In [ ]:
zero_d["SYMPTOM_TEXT" ]=zero_d['SYMPTOM_TEXT'].str.wrap(100)
zero_d['SYMPTOM_TEXT']= zero_d['SYMPTOM_TEXT'].str.replace('\\n','<br />')

# Chart sections for Age Years and died within 1 day of taking a COVID Vaccine 

In [ ]:
zbar= zero_d.groupby('AGE_YRS')['DIED'].count().to_frame().reset_index()
zbar

,AGE_YRS,DIED
0,2.0,1
1,10.0,1
2,14.0,1
3,17.0,1
4,23.0,2
...,...,...
63,95.0,5
64,96.0,2
65,97.0,2
66,100.0,2


In [ ]:
fig = make_subplots(rows=1, cols=2,shared_yaxes=False)

fig.add_trace(
    go.Box( y=zero_d['AGE_YRS'], boxpoints='all',name='Median Mortality', text=zero_d[['AGE_YRS','SYMPTOM1','SYMPTOM2','SYMPTOM3','SYMPTOM4','SYMPTOM5','SYMPTOM_TEXT']],hovertemplate = "AGE %{text[0]} <br> %{text[1]}  <br> %{text[2]} <br> %{text[3]} <br> %{text[4]} <br> %{text[5]} <br> %{text[6]}"),
    row=1, col=1
)

fig.add_trace(
    go.Bar(x=zbar['AGE_YRS'],y=zbar["DIED"],name='Total by Age',width=1, text=zbar[['AGE_YRS','DIED']],hovertemplate = "AGE %{text[0]} <br> Deaths Reported: %{text[1]}") ,
    row=1, col=2
)

fig.update_layout(legend_traceorder="normal",template='plotly_dark',legend=dict(
        itemclick="toggleothers",
        itemdoubleclick="toggle"),hoverlabel=dict(
        bgcolor="red",
        font_size=30,
        font_family="Rockwell"))

fig.update_layout(font=dict(family="Droid Sans Mono",size=30))
fig.update_yaxes(tickfont_size=30, ticks="outside", ticklen=20, tickwidth=10,showspikes=True)
fig.update_xaxes(tickfont_size=30, ticks="outside", ticklen=20, tickwidth=10,showspikes=True)
fig.update_layout(height=1000, width=1900, title_text=f"DIED or DIED WITHIN 1 DAY OF SHOT -REPORTS TO VAERS #{d_count}")
fig.show()

# Chart section for Age of years and dieing within 11-500 days of taking a COVID Vaccine 

In [ ]:
dlong = nodupids.query('NUMDAYS >=11 and NUMDAYS <500')
dlong.replace(np.nan,"N", inplace= True)
dlong["SYMPTOM_TEXT" ]=dlong['SYMPTOM_TEXT'].str.wrap(100)

In [ ]:
dlong['SYMPTOM_TEXT']= dlong['SYMPTOM_TEXT'].str.replace('\\n','<br />')

In [ ]:
fig = px.scatter(dlong, x='NUMDAYS',y='VAERS_ID',color="DIED",hover_data=['SYMPTOM_TEXT'])
fig.update_layout(height=900, width=1900, title_text=f"LONG RANGE -REPORTS TO VAERS 11 Days or More as of {date}")
fig.update_layout(legend_traceorder="normal",template='plotly_dark',legend=dict(
        itemclick="toggleothers",
        itemdoubleclick="toggle"),hoverlabel=dict(
        bgcolor="black",
        font_size=30,
        font_family="Rockwell"
    )   )
fig.update_layout(font=dict(family="Droid Sans Mono",size=30))
fig.update_xaxes(tickfont_size=30, ticks="outside", ticklen=20, tickwidth=10)
fig.update_yaxes(tickfont_size=30, ticks="outside", ticklen=20, tickwidth=10)
fig.show()

In [ ]:
# Create a figure with a single subplot
fig = make_subplots(rows=1, cols=1)

# Add a histogram trace showing the frequency of age groups reported to VAERS
fig.add_trace(
    go.Histogram(x=nodupids['AGE_YRS'], name='Age Group Frequency')
)

# Update the layout of the figure to include a title and axis labels
fig.update_layout(
    title='Frequency of Age Groups Reported to VAERS',
    xaxis_title='Age Group',
    yaxis_title='Frequency'
)

# Display the figure
fig.show()

In [ ]:
# Filter the data to include only records for individuals who have died
death_data = nodupids[nodupids['DIED'] == 'Y']

# Group the data by age group and count the number of deaths in each group
death_counts_by_age = death_data.groupby('AGE_YRS')['DIED'].count().reset_index()

# Create a figure with a single subplot
fig = make_subplots(rows=1, cols=1)

# Add a bar chart trace showing the frequency of deaths by age group
fig.add_trace(
    go.Bar(x=death_counts_by_age['AGE_YRS'], y=death_counts_by_age['DIED'], name='Death Frequency by Age Group')
)

# Update the layout of the figure to include a title and axis labels
fig.update_layout(
    title='Frequency of Deaths by Age Group',
    xaxis_title='Age Group',
    yaxis_title='Frequency'
)

# Display the figure
fig.show()

In [ ]:
# Filter the data to include only records for the 'MODERNA' vaccine manufacturer
manufacturer_data = nodupids[nodupids['VAX_MANU'] == 'MODERNA']

# Filter the data to include only records for individuals who have died
death_data = manufacturer_data[manufacturer_data['DIED'] == 'Y']

# Group the data by age group and count the number of deaths in each group
death_counts_by_age = death_data.groupby('AGE_YRS')['DIED'].count().reset_index()

# Create a figure with a single subplot
fig = make_subplots(rows=1, cols=1)

# Add a bar chart trace showing the frequency of deaths by age group
fig.add_trace(
    go.Bar(x=death_counts_by_age['AGE_YRS'], y=death_counts_by_age['DIED'], name='Death Frequency by Age Group')
)

# Update the layout of the figure to include a title and axis labels
fig.update_layout(
    title='Frequency of Deaths by Age Group and MODERNA Vaccine Manufacturer',
    xaxis_title='Age Group',
    yaxis_title='Frequency'
)

# Display the figure
fig.show()
